In [1]:

from dotenv import load_dotenv
import os
import sys
import json
import pandas as pd
import time
import datetime
load_dotenv()

experiment_path = '../..'

path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"/functions")


from dataset_utils import DatasetEvaluator

/Users/moon/Documents/text_to_sql_byLLM/experiments/C3-shipment/../..


In [2]:
DEFAULT_PATH = "results/"
EXPERIMENT = "C3_SHIPMENT"
TARGET_PATH = f"evaluator_results/{EXPERIMENT}_eval.csv" 
TARGET_PATH_FK = f"evaluator_results/{EXPERIMENT}_foreign_key_results.csv"
result_file_path = "results/c3_shipment_results.json"

In [3]:
def save_queries(queries, FILE_NAME_PATH):
    data = {"queries":queries}
    with open(FILE_NAME_PATH, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries(FILE_NAME_PATH):
    with open(FILE_NAME_PATH, encoding="utf-8", errors="ignore") as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

def get_schema(FILE_NAME_PATH):
    if "shipment" in FILE_NAME_PATH:
        return "shipment"
    elif "mondial_gpt" in FILE_NAME_PATH:
        return "mondial_gpt"
    elif "mondial" in FILE_NAME_PATH:
        return "mondial"
    return ""

def get_count_query_type(queries):
    count_types = {'simple': 0, 'medium': 0, 'complex': 0}
    for item in queries:
        type = item["type"]
        if type in count_types:
            count_types[type] += 1
        else:
            count_types[type] = 1
    count_types['total'] = len(queries)
    return count_types

def get_dataset_evaluator(FILE_NAME_PATH):
    
    dataset_eval = DatasetEvaluator(dataset_file_path=f"../../datasets/shipment/shipment_dataset.json",
                                dataset_tables_path=f"../../datasets/shipment/result_tables/",
                                db_connection_file=f"../../datasets/shipment_db_connection.json",
                                dataset_name='"shipment"',)
    return dataset_eval

def get_files_in_folder():
    files = []
    for r, d, f in os.walk(DEFAULT_PATH):
        for file in f:
            if '.json' in file and 'erro' not in file:
                files.append(os.path.join(r, file))
    return sorted(files)

def evaluate_query(queries, dataset_eval, FILE_NAME_PATH):
    result = {'simple': 0, 'medium': 0, 'complex': 0, 'total': 0}

    for q in queries:
        evaluate_result = False
        q['similarity'] = 0
        try:
            if q["query_string"].strip() != "":
                evaluate_result, similarity, column_matching_index = dataset_eval.evaluate_dataset_query(q["query_string"], q["id"], query_type="sql")
                q['column_matching_index'] = column_matching_index
                q['similarity']=similarity
            
        except Exception as e:
            print(str(e))
            
        q['result'] = evaluate_result
        
        if evaluate_result==True:
            result['total'] += 1
            result[q["type"]] += 1
        
    save_queries(queries, FILE_NAME_PATH)   
    return result

def get_file_name(FILE_NAME_PATH):
    parts = FILE_NAME_PATH.split("/")
    last_element = parts[-1]
    file_name = last_element.replace(".json", "")
    return file_name

def get_experiment_name(filename):
    schema = get_schema(filename).upper()
    experiment_name = f"{EXPERIMENT} - {schema}"
    if "chatgpt4" in filename:
        experiment_name += " - GPT-4"
    elif "chatgpt_" in filename:
        experiment_name += " - GPT-3.5"
    elif "llama" in filename:
        experiment_name += " - LLAMA"
    
    if "turbo" in filename:
        experiment_name += " - Turbo"
        
    if "instances" in filename:
        experiment_name += " - Passing Database Rows"
    elif "code_representation" in filename:
        experiment_name += " - Code Representation"
    if "_fk" in filename:
        experiment_name += " - FK"       
    return experiment_name

def compute_cost(filename):
    queries = read_queries(filename)
    total_costs = 0
    total_tokens = 0
    input_tokens = 0
    output_tokens = 0
    #total_token
    for q in queries:
        data = q.get('token_usage', None)
        if data is not None:
            for key, track_token in data.items():
                total_tokens += track_token.get('total_tokens', 0)
                total_costs += track_token.get('total_cost', 0)
                input_tokens += track_token.get('prompt_tokens', 0)
                output_tokens += track_token.get('completion_tokens', 0)
    print(total_tokens)
    print(round(total_costs, 2))

    return total_tokens, round(total_costs, 2), input_tokens, output_tokens

def compute_cost_specific(filename, specific_trackin):
    queries = read_queries(filename)
    specific_info = 0
    for q in queries:
        data = q.get('token_usage', None)
        if data is not None:
            for key, track_token in data.items():
                specific_info += track_token.get(specific_trackin, 0)
    return specific_info

def compute_time(filename):
    queries = read_queries(filename)
    total_time = 0
    for q in queries:
        total_time += q.get('time', 0)
    print(total_time)
    return str(datetime.timedelta(seconds=round(total_time)))

In [4]:
def run(result_files = [], results = [], compute_costs = True):
    
    if result_files == [] :
        result_files = get_files_in_folder()
    else:
        result_files = sorted(result_files)
        
    
    for file in result_files:
        queries = read_queries(file)
        dataset_eval = get_dataset_evaluator(file)
        count_query_type = get_count_query_type(queries)
        result = evaluate_query(queries, dataset_eval, file)
        
        overall_by_type = {}
        for item in result:
            overall_by_type[f"overall_{item}"] = round(result[item] / count_query_type[item], 4)
        
        result['experiment'] = get_file_name(file)
        result = {**result, **overall_by_type}
        
        result['total_tokens'], result['total_cost'], result['average_tokens_by_query'] = 0, 0, 0
        if compute_costs:
            result['total_tokens'], result['total_cost'], result['input_tokens'], result['output_tokens'] = compute_cost(file)
        result['total_time'] = compute_time(file)
        results.append(result)
        time.sleep(5)
    return results


In [5]:
compute_cost('results/c3_shipment_results.json')

0
0


(0, 0, 0, 0)

In [6]:
compute_cost_specific('results/c3_shipment_results.json', 'prompt_tokens') 

0

In [7]:
compute_cost_specific('results/c3_shipment_results.json', 'completion_tokens')

#Testing

# with open('../../datasets/shipment/shipment_dataset.json') as f:
#     data = json.load(f)
#     for q in data['queries']:
#         if 'query_string' not in q:
#             print(f"Missing query_string in query ID: {q.get('id', 'unknown')}")

0

In [8]:
results_fk = run([
 'results/c3_shipment_results.json'], [])

{'DB_USER_NAME': 'SHIPMENT', 'DB_PASS': 'oraclee', 'DB_HOST': 'localhost', 'DB_PORT': 1521, 'DB_NAME': 'XE', 'SQL_DRIVER': 'oracle+oracledb', 'SERVICE_NAME': 'XE', 'SCHEMA': 'SHIPMENT', 'KEYWORD_SEARCH_API_URL': ''}
'query'
'query'
'query'
'query'
'query'
'query'
'query'
'query'
'query'
'query'


ZeroDivisionError: division by zero

In [ ]:
df_fk = pd.DataFrame(results_fk, columns=['experiment', 'simple', 'medium', 'complex', 'total', 'overall_simple', 'overall_medium', 'overall_complex', 'overall_total','input_tokens', 'output_tokens', 'total_tokens','total_cost', 'total_time'])
df_fk['experiment'] = df_fk['experiment'].apply(get_experiment_name)
df_fk = df_fk.sort_values(by='overall_total', ascending=False)
df_fk

In [ ]:
df_fk.to_csv(TARGET_PATH_FK)  